In [ ]:
import os
from glob import glob
import random
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cv2 as cv

Button

In [ ]:
test_type = 'elec'
avg_hum = True
base_path = f'[USER_PATH]\\ArtificialVision'

num_class = 16
if test_type == 'opt':
  sel_ppl = list(range(300, 309)) + list(range(400, 408)) + [611]
  sel_mc = [1, 2, 3]
elif test_type == 'elec':
  sel_ppl = [499, 500] + list(range(502, 509)) + list(range(601, 607)) + list(range(608, 611)) 
  sel_mc = [4, 5, 6]

quesdata_files = glob(f'{base_path}\\data\\Question_Banks_AI_Hub_final\\16classes\\set0\\*')
humdata_path = glob(f'{base_path}\\data\\Human_Expert\\211202')[0]
humsub_path = glob(f'{base_path}\\data\\210827_ANNA_Removing_uncontaminated_data.csv')[0]
answer_path = glob(f'{base_path}\\data\\211105_QAs_for_Set0_CNN_SVC_4classes_partial.csv')[0]
testdata_path = glob(f'{base_path}\\data\\sample_for_dev_test_{test_type}')[0]
fig_path = glob(f'{base_path}\\figures')[0]

Human Data

In [ ]:
human_df = pd.DataFrame()
n = 9
for i in range(1, 80*n+1, 80):
    try:
      j = i+79
      temp_df = pd.read_csv(os.path.join(humdata_path, f'main_test({i}_{j}).csv'))
      if i == 1:
        pass
      else:
        temp_df = temp_df.rename(columns = {'유저식별아이디':'useless', 'MC구분':'useless', '성별':'useless', '나이':'useless', '학력':'useless'})
      human_df = pd.concat([human_df, temp_df], axis=1)
    except:
      print(i)

human_df = human_df[human_df['유저식별아이디'].isin(sel_ppl)]

if avg_hum:
  human_df = human_df[human_df['MC구분'].isin(sel_mc)]
orig_human_df = human_df
orig_human_df

Answer Data

In [ ]:
answer_df = pd.read_csv(answer_path)

act_per_list, pix_list, gs_list, par_list = [], [], [], []
for answer in answer_df['Answer']:
    img, _ = answer.split('.jpg')
    act_per, pix, gs, par = img.split('_')
    
    act_per_list.append(act_per)
    pix_list.append(pix)
    gs_list.append(gs)
    par_list.append(par)

answer_df['act_per'] = act_per_list
answer_df['PIX'] = pix_list
answer_df['GS'] = gs_list
answer_df['par'] = par_list

answer_df = answer_df[:80*n]
answer_df = answer_df.T

orig_answer_df = answer_df
answer_df

Preprocessed Human Data

In [ ]:
human_df = orig_human_df.copy()
human_df = human_df.fillna(0)
if avg_hum:
    human_df['MC구분'] = human_df['MC구분'].astype(int)
    human_df.index = human_df['MC구분']
else:
    human_df['유저식별아이디'] = human_df['유저식별아이디'].astype(int)
    human_df.index = human_df['유저식별아이디']

sel_col = []
for (i, col) in enumerate(human_df.columns):
    split = col.split('_')
    if split[0] == '선택' and split[1] == 'A':
        sel_col.append(col)
    
acc_df = human_df[sel_col]
acc_df.columns = [n for n in range(acc_df.shape[1])]
acc_df = acc_df.astype(int)

for q in range(acc_df.shape[1]):
    act_per = answer_df.loc['act_per'][q] 
    for s in range(acc_df.shape[0]):
        pred_per = acc_df.iloc[s, q]
        try:
            if str(int(pred_per)) == str(int(act_per)):
                acc_df.iloc[s, q] = 1
            else:
                acc_df.iloc[s, q] = 0
        except:
            acc_df.iloc[s, q] = 0

acc_df.columns = answer_df.T['Answer']

if avg_hum:
    acc_df = acc_df.reset_index()
    acc_df = acc_df.groupby('MC구분').mean()

acc_df

Machine Data

In [ ]:
model_type1_list = [''] # ['PCA', 'PCA', '', '']
model_type2_list = ['CNN_SVC'] # ['SVC', 'LR', 'CNN_LR', 'CNN_SVC']
seed_list = [22, 77, 2, 100, 81, 42, 7, 1, 55, 50] 
pix_order_list = ['16PIX', '24PIX', '32PIX', '64PIX', '128PIX']
gs_order_list = ['2GS', '4GS', '6GS', '8GS', '16GS']

df = pd.read_csv(humsub_path)
l = list(range(df.shape[0]))
n = 16
random.seed(22)
set_1 = random.sample(l, n)
sets = [set_1]


for (model_type1, model_type2) in zip(model_type1_list, model_type2_list):
    model_type = model_type1 + model_type2
    input_folder = [df.iloc[i, 0] for i in sets[0]] 
    assert len(input_folder) == 16
    com_obj = itertools.combinations(input_folder, 16)
    com_list = list(com_obj)

    mac_df = pd.DataFrame()
    for i in range(len(quesdata_files)):
        data_path = quesdata_files[i]
        seed = int(os.path.basename(data_path).split('d')[-1])

        for n in range(len(os.listdir(data_path))):
            print(seed, n)

            preprocessed_data_path =  os.path.join(data_path, f'comb{n}') 
            high_analysis_path = os.path.join(preprocessed_data_path, f'High_Analysis_{test_type}')
            
            add_high_df = pd.read_csv(os.path.join(high_analysis_path, f'High_Level_Data_Analysis_{model_type1}_{model_type2}.csv'))
            add_high_df['Hit Rate'] = add_high_df['correctness'].replace(['correct', 'wrong'], [1, 0]) 
            add_high_df = add_high_df[['file_name', 'actual_person', 'Hit Rate']]
            add_high_df['actual_person'] = add_high_df['actual_person'].astype(int)
            
            img_list, hyperpar_list, par_list = [], [], []
            for i in range(add_high_df.shape[0]):
                file_name = add_high_df['file_name'][i].split('.')[0]
                pix, gs, par = file_name.split('_')
                per = add_high_df['actual_person'][i]
                img = str(per) + '_' + str(pix) + '_' + str(gs) + '_' + str(par) + '.jpg'
                img_list.append(img)
                hyperpar_list.append(str(pix) + '_' + str(gs))
                par_list.append(str(par))
            add_high_df['img'] = img_list
            add_high_df['hyperpar'] = hyperpar_list
            add_high_df['par'] = par_list
            
            add_high_df = add_high_df[['img', 'hyperpar', 'par', 'Hit Rate']]
        
        add_high_df['Seed'] = [seed] * add_high_df.shape[0]
        
        mac_df = pd.concat([mac_df, add_high_df], axis=0)

new_hyperpar_name_list = ['16PIX_2GS', '16PIX_4GS', '16PIX_8GS', 
                          '32PIX_2GS', '32PIX_4GS', '32PIX_8GS',  
                          '64PIX_2GS', '64PIX_4GS', '64PIX_8GS']
par_list = ['S001L1E01C4', 'S001L1E01C7', 'S001L1E01C10',
            'S001L1E02C7',
            'S001L1E03C7']
mac_df = mac_df[mac_df['hyperpar'].isin(new_hyperpar_name_list)]
mac_df = mac_df[mac_df['par'].isin(par_list)]

mac_df = mac_df.pivot(index='Seed', columns='img', values='Hit Rate')

mac_df

Model Performances Data

In [ ]:
model_type1_list = ['', '', '', '', '', '', '', '', '', '', 'PCA', 'PCA'] 
model_type2_list = ['PIXEL_LR','PIXEL_SVC', 'CNN_LR', 'CNN_SVC', 'CNN_VggNet2', 'CNN_VggNet2_SVC', 'CNN_AlexNet2', 'CNN_AlexNet2_SVC', 'CNN_ResNet2', 'CNN_ResNet2_SVC', 'SVC', 'LR']

for (model_type1, model_type2) in zip(model_type1_list, model_type2_list):
    model_type = model_type1 + model_type2

    high_df = pd.DataFrame()
    for i in range(len(quesdata_files)):
        data_path = quesdata_files[i]
        preprocessed_data_path =  os.path.join(data_path, 'comb0') 

        high_analysis_path = os.path.join(preprocessed_data_path, f'High_Analysis_{test_type}')
        try:
            add_high_df = pd.read_csv(os.path.join(high_analysis_path, f'High_Level_Data_Analysis_{model_type1}_{model_type2}.csv'))
            add_high_df['hit_rate'] = add_high_df['correctness'].replace(['correct', 'wrong'], [1, 0]) 
            high_df = pd.concat([high_df, add_high_df], axis=0)
            high_df = high_df[['file_name', 'actual_person', 'hit_rate']]
        except:
            print(model_type, i, 'error')
            pass
            
    image_list = []
    for i, file_name in enumerate(high_df['file_name']):
        face = high_df['actual_person'].iloc[i]
        face = str(int(float(face)))
        image_list.append(f'{face}_{file_name}')
    high_df['image'] = image_list

    high_df = high_df[['image', 'hit_rate']]

    old_high_df_cols = high_df.columns
    new_high_df_cols = [f'{model_type}_' + col_name for (i, col_name) in enumerate(old_high_df_cols) if i != 0] 
    new_high_df_cols = [old_high_df_cols.tolist()[0], *new_high_df_cols]
    high_df.columns = new_high_df_cols

    high_df = pd.DataFrame(high_df.groupby('image').mean().reset_index()) 
    try:
        assert high_df.shape[0] == 3600
    except:
        print(data_path)

    if model_type == 'PCALR':
        pca_lr_high_df = high_df
    elif model_type == 'PCASVC':
        pca_svc_high_df = high_df
    elif model_type == 'PCASVC2':
        pca_svc2_high_df = high_df
    elif model_type == 'CNN_LR':
        cnn_lr_high_df = high_df
    elif model_type == 'CNN_SVC':
        cnn_svc_high_df = high_df
    elif model_type == 'CNN_VggNet2':
        cnn_vgg_high_df = high_df
    elif model_type == 'CNN_VggNet2_SVC':
        cnn_vgg_svc_high_df = high_df
    elif model_type == 'CNN_AlexNet2':
        cnn_alexnet_high_df = high_df
    elif model_type == 'CNN_AlexNet2_SVC':
        cnn_alexnet_svc_high_df = high_df
    elif model_type == 'CNN_ResNet2':
        cnn_resnet_high_df = high_df
    elif model_type == 'CNN_ResNet2_SVC':
        cnn_resnet_svc_high_df = high_df
    elif model_type == 'PIXEL_SVC':
        pixel_svc_high_df = high_df
    elif model_type == 'PIXEL_LR':
        pixel_lr_high_df = high_df

model_list = [pixel_svc_high_df, pixel_lr_high_df,
              pca_svc_high_df, pca_lr_high_df,
              cnn_svc_high_df, cnn_lr_high_df,
              cnn_alexnet_svc_high_df, cnn_alexnet_high_df,
              cnn_vgg_svc_high_df, cnn_vgg_high_df,
              cnn_resnet_svc_high_df, cnn_resnet_high_df]
model_name_list = ['PIXEL_SVC', 'PIXEL_LR',
                   'PCASVC', 'PCALR',
                   'CNN_SVC', 'CNN_LR',
                   'CNN_AlexNet2_SVC', 'CNN_AlexNet2',
                   'CNN_VggNet2_SVC', 'CNN_VggNet2',
                   'CNN_ResNet2_SVC', 'CNN_ResNet2']
macs_df = model_list[0].merge(model_list[1], on='image')
for i in range(2, len(model_list)):
    macs_df = macs_df.merge(model_list[i], on='image')

macs_df = macs_df.drop(macs_df.tail(1).index)
macs_df = macs_df.set_index('image')
macs_df.columns = model_name_list
macs_df = macs_df.transpose()
macs_df['19092711_64PIX_8GS_S001L1E03C7.jpg'] = macs_df['19092711_64PIX_8GS_S001L1E02C7.jpg'].values
macs_df

Merged Data

In [ ]:
chosen_hyperpar = '16PIX_2GS'

temp_mer_df = pd.concat([acc_df, macs_df], join='inner')

temp_mer_df = temp_mer_df.T.astype(float)
merged_df = temp_mer_df.copy()


temp_index = []
pix_list = []
h_list = []
prepar_list = []

for file_name in merged_df.index:
    split = file_name.split('.')
    face, pix, gs, par = split[0].split('_')
    temp_index.append(f'{pix}_{gs}_{face}_{par}') # f'{pix}_{gs}_{face}_{par}'
    pix_list.append(pix)
    h_list.append(f'{pix}_{gs}')

merged_df['idx'] = temp_index
merged_df['PIX'] = pix_list
merged_df['hyperpar'] = h_list
if chosen_hyperpar != '':
    new_merged_df = merged_df[merged_df['hyperpar'] == chosen_hyperpar]
else:
    new_merged_df = merged_df

temp_df = new_merged_df.groupby('idx').mean()

temp_df = temp_df.sort_index()

new_temp_df = temp_df.reset_index()

for i in range(4, temp_df.shape[0], 5):
    new_temp_df.loc[i+0.1] = [0]*new_temp_df.shape[1]
    new_temp_df.loc[i+0.2] = [0]*new_temp_df.shape[1]
new_temp_df = new_temp_df.sort_index().reset_index(drop=True)

new_temp_df.index = new_temp_df['idx']
new_temp_df = new_temp_df.iloc[:, 1:]
new_temp_df

Color Dictionary

In [ ]:
color_index_dict = {k:[] for k in range(1, new_temp_df.shape[1]+1)}
for (t, col) in enumerate(new_temp_df.columns):
    temp_col = new_temp_df[col]
    color_index_dict[t+1] = temp_col.values

Results of Color Maps

In [ ]:
if avg_hum:
    l = len(sel_mc)
else:
    l = len(sel_ppl)
dim = 2
col_interval = 2

gridspec = dict(wspace=0.0, width_ratios=[1, col_interval, *[1]*l, col_interval, *[1]*(new_temp_df.shape[1]-l)])
fig, ax = plt.subplots(new_temp_df.shape[0], new_temp_df.shape[1]+3, figsize=(dim*(new_temp_df.shape[1]+3), dim*new_temp_df.shape[0]), gridspec_kw=gridspec)
for i, idx in enumerate(new_temp_df.index):

    if (idx == 0):
        for j in range(new_temp_df.shape[1]+3):
            ax[i][j].imshow(np.ones((*img.shape, 3))*255, cmap='gray')
            ax[i][j].set_xticks([])
            ax[i][j].set_yticks([])
            ax[i][j].spines['top'].set_color('white')
            ax[i][j].spines['bottom'].set_color('white')
            ax[i][j].spines['left'].set_color('white')
            ax[i][j].spines['right'].set_color('white')
            ax[i][j].set_aspect('equal')

    else:
        if chosen_hyperpar == '':
            par = idx
        else:
            pix, gs, face, par = idx.split('_') 
            # person = '19082131'

        filePath = os.path.join(os.path.join(testdata_path, face), f'{pix}_{gs}_{par}.jpg')
        assert os.path.isfile(filePath)
        img = cv.imread(filePath, cv.IMREAD_GRAYSCALE)
        for j in range(new_temp_df.shape[1]+3):
            if j == 0:
                ax[i][0].imshow(np.asarray(img), cmap='gray')
            elif (j == 1 or j == l+2):
                ax[i][1].imshow(np.ones((*img.shape, 3))*255, cmap='gray')
            else:
                if (j >= 2) and (j <= l-1):
                    hit_rate = color_index_dict[j-1][i]
                else:
                    hit_rate = color_index_dict[j-2][i]
                num_val = 1000
                assert hit_rate >= 0 and hit_rate <= 1
                conv_hr = hit_rate 
                conv_hr = conv_hr * (num_val - 1) 
                conv_hr = round(conv_hr)

                new_conv_hr = conv_hr
                color = sns.color_palette('Spectral', num_val)[new_conv_hr] 
                
                ax[i][j].set_facecolor(color)
                
            ax[i][j].set_xticks([])
            ax[i][j].set_yticks([])
            ax[i][j].spines['top'].set_color('white')
            ax[i][j].spines['bottom'].set_color('white')
            ax[i][j].spines['left'].set_color('white')
            ax[i][j].spines['right'].set_color('white')
            ax[i][j].set_aspect('equal')

fig.subplots_adjust(wspace=0, hspace=0)
plt.savefig(os.path.join(fig_path, f'face_{chosen_hyperpar}_{test_type}.png'))

Correlations

In [ ]:
corr = temp_df.corr()
corr.to_csv(os.path.join(fig_path, f'corr_{test_type}_{chosen_hyperpar}.csv'))
temp_df.to_csv(os.path.join(fig_path, f'scat_{test_type}_{chosen_hyperpar}.csv'))

In [ ]:
sns.heatmap(corr)

Results of PIX & GS

In [ ]:
pixgs_df = macs_df.copy()

hyp_lst = []
for img in macs_df.index:
    _, pix, gs, _ = img.split('_')
    hyp_lst.append(f'{pix}_{gs}')

pixgs_df['hyp'] = hyp_lst
pixgs_df = pixgs_df.groupby('hyp').agg(['mean', 'sem'])

pix_lst, gs_lst = [], []
for img in pixgs_df.index:
    pix, gs = img.split('_')
    pix_lst.append(pix)
    gs_lst.append(gs)

pixgs_df['PIX'] = pix_lst
pixgs_df['GS'] = gs_lst

pixgs_df

In [ ]:
pixgs_df.to_csv(os.path.join(fig_path, f'pixgs_{test_type}.csv'))